In [1]:
import tensorflow_probability as tfp
tfd = tfp.distributions
import pandas as pd
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy import stats

In [2]:
# Load the California Housing dataset
housing = fetch_california_housing(as_frame=True)


df = pd.DataFrame(data=housing.data, columns=housing.feature_names)
df["target"] = housing.target

df = df[(np.abs(stats.zscore(df)) < 5).all(axis=1)]
df = df.reset_index(drop=True)
X = df[housing.feature_names]
targets = df["target"]
# Always use transform_data function on X
scaler = StandardScaler().fit(np.array(targets).reshape(-1, 1))
targets = scaler.transform(np.array(targets).reshape(-1, 1)).flatten()

df["target"] = targets
data=df

In [3]:
import sys
sys.path.append('..')
from xDL.models.NAMLSS import NAMLSS

In [4]:
namlss = NAMLSS(
    "target ~  -1 + MLP(MedInc) + MLP(AveOccup) + MLP(AveBedrms)+ MLP(Population)+  MLP(Latitude):MLP(Longitude) + MLP(AveRooms)", 
    data=data, 
    family="Normal", 
    feature_dropout=0.0001
    )


--- Preprocessing ---


8it [00:04,  1.66it/s]
7it [00:00, ?it/s]


------------- Network architecture --------------
chosen distribution: Normal, distributional parameters: ['loc', 'scale']
MedInc -> MLP(feature=MedInc, n_params=12800) -> output dimension=2
AveOccup -> MLP(feature=AveOccup, n_params=12800) -> output dimension=2
AveBedrms -> MLP(feature=AveBedrms, n_params=12800) -> output dimension=2
Population -> MLP(feature=Population, n_params=12800) -> output dimension=2
Latitude_._Longitude -> MLP(feature=Latitude_._Longitude, n_params=12928) -> output dimension=2
AveRooms -> MLP(feature=AveRooms, n_params=12800) -> output dimension=2


In [5]:
def KL_Divergence(y_true, y_hat):
    t = tfd.Normal(loc=y_true, scale=tf.math.reduce_std(y_true))
    p = tfd.Normal(loc=y_hat, scale=tf.math.reduce_std(y_true))  
    kl = tf.reduce_mean(tfd.kl_divergence(t, p, allow_nan_stats=True))
    return kl

namlss.compile(
    optimizer=Adam(learning_rate=0.001), 
    loss=namlss.NegativeLogLikelihood, 
    metrics=KL_Divergence
    )

# Train the model
namlss.fit(namlss.training_dataset, epochs=100, validation_data=namlss.validation_dataset)

# Evaluate the model
loss = namlss.evaluate(namlss.validation_dataset)
print("Test Loss:", loss)

Epoch 1/100
16/16 [==============================] - 9s 127ms/step - loss: 2.2432 - KL_Divergence: 1.3859 - val_loss: 1.3457 - val_KL_Divergence: 1.1865
Epoch 2/100
16/16 [==============================] - 1s 45ms/step - loss: 9725.6211 - KL_Divergence: 1.4308 - val_loss: 1.2566 - val_KL_Divergence: 0.8791
Epoch 3/100
16/16 [==============================] - 1s 48ms/step - loss: 1.3531 - KL_Divergence: 1.0739 - val_loss: 1.1431 - val_KL_Divergence: 0.7732
Epoch 4/100
16/16 [==============================] - 1s 43ms/step - loss: 1.2829 - KL_Divergence: 0.8838 - val_loss: 1.0841 - val_KL_Divergence: 0.6798
Epoch 5/100
16/16 [==============================] - 1s 44ms/step - loss: 1.1915 - KL_Divergence: 0.7871 - val_loss: 1.0377 - val_KL_Divergence: 0.6368
Epoch 6/100
16/16 [==============================] - 1s 35ms/step - loss: 1.4952 - KL_Divergence: 0.7824 - val_loss: 1.0332 - val_KL_Divergence: 0.6214
Epoch 7/100
16/16 [==============================] - 1s 40ms/step - loss: 1.1427 - K

In [6]:
namlss.analytics_plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [7]:
namlss.plot_dist()

100/100 [==============================] - 0s 1ms/step


In [8]:
namlss.plot()